In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List
import os
from utils.datapipe import to_snake_case, DataPipe
from utils.helper_funcs import unzip_directory
import requests
from bs4 import BeautifulSoup as BS
import re
from urllib.request import urlretrieve

plt.style.use('ggplot')

This notebook explores all 4 sets of data from Freddie Mac: 
1. non-standard origination data
2. standard origination data
3. non-standard time-series data
4. standard time-series data

The non-standard origination data and standard origination data will be combined--like wise for the time series data.

In [5]:
# dummy model = classification where you predict using a binom rvs where the threshold
# is the class %
# what metric = profit curves

# /home/austin/code/dsi/freddiemac_mortage_defaults/data
# nonstandard_mortgage_data/historical_data_excl_2005/historical_data_excl_2005Q1.zip

In [4]:
unzip_directory('../data/')

../data/standard_mortgage_data/historical_data_2000/historical_data_2000Q4.zip
../data/standard_mortgage_data/historical_data_2000/historical_data_2000Q2.zip
../data/standard_mortgage_data/historical_data_2000/historical_data_2000Q1.zip
../data/standard_mortgage_data/historical_data_2000/historical_data_2000Q3.zip
../data/standard_mortgage_data/historical_data_2018/historical_data_2018Q1.zip
../data/standard_mortgage_data/historical_data_2018/historical_data_2018Q2.zip
../data/standard_mortgage_data/historical_data_2018/historical_data_2018Q3.zip
../data/standard_mortgage_data/historical_data_2018/historical_data_2018Q4.zip
../data/standard_mortgage_data/historical_data_2008/historical_data_2008Q3.zip
../data/standard_mortgage_data/historical_data_2008/historical_data_2008Q2.zip
../data/standard_mortgage_data/historical_data_2008/historical_data_2008Q4.zip
../data/standard_mortgage_data/historical_data_2008/historical_data_2008Q1.zip
../data/standard_mortgage_data/historical_data_2003/

In [2]:
origination_headers = ['CREDIT SCORE', 'FIRST PAYMENT DATE', 'FIRST TIME HOMEBUYER FLAG'
                      ,'MATURITY DATE', 'METROPOLITAN STATISTICAL AREA (MSA) OR METROPOLITAN DIVISION'
                      , 'MORTGAGE INSURANCE PERCENTAGE (MI %)', 'NUMBER OF UNITS'
                      , 'OCCUPANCY STATUS', 'ORIGINAL COMBINED LOAN-TO-VALUE (CLTV)'
                      , 'ORIGINAL DEBT-TO-INCOME (DTI) RATIO', 'ORIGINAL UPB'
                      , 'ORIGINAL LOAN-TO-VALUE (LTV)', 'ORIGINAL INTEREST RATE'
                      , 'CHANNEL', 'PREPAYMENT PENALTY MORTGAGE (PPM) FLAG', 'AMORTIZATION TYPE'
                      , 'PROPERTY STATE', 'PROPERTY TYPE', 'POSTAL CODE', 'LOAN SEQUENCE NUMBER'
                      , 'LOAN PURPOSE', 'ORIGINAL LOAN TERM', 'NUMBER OF BORROWERS'
                      , 'SELLER NAME', 'SERVICER NAME', 'SUPER CONFORMING FLAG'
                      , 'Pre-HARP LOAN SEQUENCE NUMBER', 'PROGRAM INDICATOR', 'HARP INDICATOR'
                      , 'PROPERTY VALUATION METHOD', 'INTEREST ONLY INDICATOR (I/O INDICATOR)'
                      ]
time_series_headers = ['LOAN SEQUENCE NUMBER', 'MONTHLY REPORTING PERIOD'
                      ,'CURRENT ACTUAL UPB', 'CURRENT LOAN DELINQUENCY STATUS'
                      , 'LOAN AGE', 'REMAINING MONTHS TO LEGAL MATURITY'
                      , 'REPURCHASE FLAG', 'MODIFICATION FLAG'
                      , 'ZERO BALANCE CODE', 'CURRENT INTEREST RATE'
                      , 'CURRENT DEFERRED UPB', 'DUE DATE OF LAST PAID INSTALLMENT (DDLPI)'
                      , 'MI RECOVERIES', 'NET SALES PROCEEDS', 'NON MI RECOVERIES'
                      , 'EXPENSES', 'LEGAL COSTS', 'MAINTENANCE AND PRESERVATION COSTS'
                      , 'TAXES AND INSURANCE', 'MISCELLANEOUS EXPENSES'
                      , 'ACTUAL LOSS CALCULATION', 'MODIFICATION COST'
                      , 'STEP MODIFICATION FLAG', 'DEFERRED PAYMENT PLAN'
                      , 'ESTIMATED LOAN TO VALUE (ELTV)', 'ZERO BALANCE REMOVAL UPB'
                      , 'DELINQUENT ACCRUED INTEREST', 'DELINQUENCY DUE TO DISASTER'
                      , 'BORROWER ASSISTANCE STATUS CODE', 'CURRENT MONTH MODIFICATION COST'
                      ]

time_series_headers = to_snake_case(time_series_headers)
origination_headers = to_snake_case(origination_headers)

headers_dict = {'origination': origination_headers
               , 'time_series': time_series_headers}

In [8]:
dir_ = '../data'
for path_object in os.walk(dir_):
    folder = path_object[0]
    for file in path_object[2]:
        print(folder + '/' + file)

../data/historical_data_1999.txt
../data/origination_data/historical_data_2019Q3.txt
../data/origination_data/historical_data_excl_2019Q4.txt
../data/origination_data/historical_data_2019Q4.txt
../data/origination_data/historical_data_excl_2019Q2.txt
../data/origination_data/historical_data_excl_2019Q1.txt
../data/origination_data/historical_data_excl_2019Q3.txt
../data/origination_data/historical_data_2019Q1.txt
../data/origination_data/historical_data_2019Q2.txt
../data/time_series_data/historical_data_excl_time_2019Q3.txt
../data/time_series_data/historical_data_time_2019Q3.txt
../data/time_series_data/historical_data_time_2019Q1.txt
../data/time_series_data/historical_data_excl_time_2019Q1.txt
../data/time_series_data/historical_data_time_2019Q2.txt
../data/time_series_data/historical_data_excl_time_2019Q2.txt
../data/time_series_data/historical_data_excl_time_2019Q4.txt
../data/time_series_data/historical_data_time_2019Q4.txt


In [9]:
datapipe = DataPipe('../data/', headers_dict)

<generator object walk at 0x7f7a2fedae50>
('../data/', ['standard_mortgage_data', 'nonstandard_mortgage_data', 'origination_data', 'time_series_data'], ['historical_data_1999.txt'])
('../data/standard_mortgage_data', [], [])
('../data/nonstandard_mortgage_data', [], [])
('../data/origination_data', [], ['historical_data_2019Q3.txt', 'historical_data_excl_2019Q4.txt', 'historical_data_2019Q4.txt', 'historical_data_excl_2019Q2.txt', 'historical_data_excl_2019Q1.txt', 'historical_data_excl_2019Q3.txt', 'historical_data_2019Q1.txt', 'historical_data_2019Q2.txt'])


/home/austin/anaconda3/envs/dsienv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (26,28) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


('../data/time_series_data', [], ['historical_data_excl_time_2019Q3.txt', 'historical_data_time_2019Q3.txt', 'historical_data_time_2019Q1.txt', 'historical_data_excl_time_2019Q1.txt', 'historical_data_time_2019Q2.txt', 'historical_data_excl_time_2019Q2.txt', 'historical_data_excl_time_2019Q4.txt', 'historical_data_time_2019Q4.txt'])


/home/austin/anaconda3/envs/dsienv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (7,23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/austin/anaconda3/envs/dsienv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (3,7,23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
datapipe.raw_data

{'time_series': Empty DataFrame
 Columns: []
 Index: [],
 'origination': Empty DataFrame
 Columns: []
 Index: []}

In [11]:
ns_time_series_df = pd.read_csv('../data/time_series_data/historical_data_excl_time_2019Q1.txt'
                         , sep='|', names=time_series_headers, index_col=False)

In [12]:
ns_time_series_df.head()

,loan_sequence_number,monthly_reporting_period,current_actual_upb,current_loan_delinquency_status,loan_age,remaining_months_to_legal_maturity,repurchase_flag,modification_flag,zero_balance_code,current_interest_rate,...,actual_loss_calculation,modification_cost,step_modification_flag,deferred_payment_plan,estimated_loan_to_value_(eltv),zero_balance_removal_upb,delinquent_accrued_interest,delinquency_due_to_disaster,borrower_assistance_status_code,current_month_modification_cost
0,A19Q10000001,201902,180000.0,0,0,360,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,49.0,NaN,NaN,NaN,NaN
1,A19Q10000001,201903,180000.0,0,1,359,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,47.0,NaN,NaN,NaN,NaN
2,A19Q10000001,201904,180000.0,0,2,358,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,47.0,NaN,NaN,NaN,NaN
3,A19Q10000001,201905,179000.0,0,3,357,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,46.0,NaN,NaN,NaN,NaN
4,A19Q10000001,201906,179000.0,0,4,356,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,46.0,NaN,NaN,NaN,NaN


In [13]:
ns_time_series_df['current_loan_delinquency_status'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16])

In [17]:
ns_time_series_df.tail()

,loan_sequence_number,monthly_reporting_period,current_actual_upb,current_loan_delinquency_status,loan_age,remaining_months_to_legal_maturity,repurchase_flag,modification_flag,zero_balance_code,current_interest_rate,...,actual_loss_calculation,modification_cost,step_modification_flag,deferred_payment_plan,estimated_loan_to_value_(eltv),zero_balance_removal_upb,delinquent_accrued_interest,delinquency_due_to_disaster,borrower_assistance_status_code,current_month_modification_cost
77648,F19Q10281674,202102,192554.98,0,23,338,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,54.0,NaN,NaN,NaN,NaN
77649,F19Q10281674,202103,192252.44,0,24,337,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,54.0,NaN,NaN,NaN,NaN
77650,F19Q10281680,202101,230355.17,4,21,339,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77651,F19Q10281680,202102,230355.17,5,22,338,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,49.0,NaN,NaN,NaN,F
77652,F19Q10281680,202103,230355.17,6,23,337,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,49.0,NaN,NaN,Y,F


In [18]:
ns_time_series_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77653 entries, 0 to 77652
Data columns (total 30 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   loan_sequence_number                       77653 non-null  object 
 1   monthly_reporting_period                   77653 non-null  int64  
 2   current_actual_upb                         77653 non-null  float64
 3   current_loan_delinquency_status            77653 non-null  int64  
 4   loan_age                                   77653 non-null  int64  
 5   remaining_months_to_legal_maturity         77653 non-null  int64  
 6   repurchase_flag                            2345 non-null   object 
 7   modification_flag                          4 non-null      object 
 8   zero_balance_code                          2345 non-null   float64
 9   current_interest_rate                      2345 non-null   float64
 10  current_deferred_upb  

In [20]:
for col in ns_time_series_df:
    print(ns_time_series_df[col].value_counts())

A19Q10000962    26
A19Q10000305    26
A19Q10000298    26
A19Q10000109    26
A19Q10000043    26
                ..
F19Q10096034     1
A19Q10000747     1
A19Q10002209     1
A19Q10001970     1
A19Q10000899     1
Name: loan_sequence_number, Length: 4087, dtype: int64
201905    3969
201906    3954
201907    3937
201904    3930
201908    3901
201909    3843
201910    3749
201911    3626
201912    3551
202001    3469
202002    3404
202003    3322
202004    3185
202005    3052
202006    2896
202007    2739
202008    2628
202009    2500
202010    2383
201903    2270
202011    2263
202012    2176
202101    2068
202102    1974
202103    1866
201902     998
Name: monthly_reporting_period, dtype: int64
0.00         2345
199000.00     229
149000.00     164
248000.00     149
200000.00     147
             ... 
284913.31       1
258973.69       1
124739.44       1
351269.94       1
231538.42       1
Name: current_actual_upb, Length: 44090, dtype: int64
0     75774
1       629
2       275
3       210
4